## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.7 MB/s 


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.7.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [11]:
train_images.shape

(60000, 28, 28, 1)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 52s]
val_accuracy: 0.9096666574478149

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 06m 23s
INFO:tensorflow:Oracle triggered exit


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        55360     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 64)                2359360   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,416,330
Trainable params: 2,416,330
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=9, validation_split=0.1, initial_epoch=6)

Epoch 4/6
1688/1688 [==============================] - 18s 11ms/step - loss: 0.1318 - accuracy: 0.9514 - val_loss: 0.2848 - val_accuracy: 0.9125
Epoch 5/6
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0931 - accuracy: 0.9647 - val_loss: 0.2988 - val_accuracy: 0.9125
Epoch 6/6
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0650 - accuracy: 0.9767 - val_loss: 0.3588 - val_accuracy: 0.9128


In [20]:
model.fit(train_images, train_labels, epochs=9, validation_split=0.1, initial_epoch=6)

Epoch 7/9
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0473 - accuracy: 0.9830 - val_loss: 0.4366 - val_accuracy: 0.9077
Epoch 8/9
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0379 - accuracy: 0.9856 - val_loss: 0.4918 - val_accuracy: 0.9067
Epoch 9/9
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0274 - accuracy: 0.9908 - val_loss: 0.5285 - val_accuracy: 0.9033
